(notebook exécuté sur Google Colab)

In [ ]:
from google.colab import drive


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install sktime

# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 10.1.6
- **Date :** 18/11/2023
- **Heure :** 11:00
- **Données :** `model_weatherAUS.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** `StandardScaler`
- **Rééquilibrage :** `RandomUnderSampler`
- **Conversion :** `numpyfy`
- **Algorithme :** KNN + DTW

## Sommaire

1. Initialisation
2. Découpage
3. Mise à l'échelle
4. Rééquilibrage
5. Conversion
6. Modélisation
7. Évaluation

## 1. Initialisation

In [ ]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [ ]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/projet_colab/model_weatherAUS.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [ ]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,Month,Day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum,NonMesNum
Date,,,,,,,,,,,,,,,,,,,,,
2007-11-01,8.0,24.3,0.0,3.4,6.3,30.0,6.0,20.0,68.0,29.0,...,11,1,15,2.356194,3.926991,2.356194,-35.297591,149.101268,1,2
2007-11-02,14.0,26.9,3.6,4.4,9.7,39.0,4.0,17.0,80.0,36.0,...,11,2,15,0.392699,0.000000,3.141593,-35.297591,149.101268,1,2
2007-11-03,13.7,23.4,3.6,5.8,3.3,85.0,6.0,6.0,82.0,69.0,...,11,3,15,2.356194,1.570796,1.178097,-35.297591,149.101268,1,2
2007-11-04,13.3,15.5,39.8,7.2,9.1,54.0,30.0,24.0,62.0,56.0,...,11,4,15,2.356194,2.748894,3.141593,-35.297591,149.101268,1,2
2007-11-05,7.6,16.1,2.8,5.6,10.6,50.0,20.0,28.0,68.0,49.0,...,11,5,15,5.105088,5.105088,5.890486,-35.297591,149.101268,1,2


In [ ]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140787 entries, 2007-11-01 to 2017-06-25
Data columns (total 29 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   MinTemp         140787 non-null  float64
 1   MaxTemp         140787 non-null  float64
 2   Rainfall        140787 non-null  float64
 3   Evaporation     140787 non-null  float64
 4   Sunshine        140787 non-null  float64
 5   WindGustSpeed   140787 non-null  float64
 6   WindSpeed9am    140787 non-null  float64
 7   WindSpeed3pm    140787 non-null  float64
 8   Humidity9am     140787 non-null  float64
 9   Humidity3pm     140787 non-null  float64
 10  Pressure9am     140787 non-null  float64
 11  Pressure3pm     140787 non-null  float64
 12  Cloud9am        140787 non-null  float64
 13  Cloud3pm        140787 non-null  float64
 14  Temp9am         140787 non-null  float64
 15  Temp3pm         140787 non-null  float64
 16  RainToday       140787 non-null  int64  
 17  Ra

## 2. Découpage

In [ ]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [ ]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Mise à l'échelle

In [ ]:
# Mise à l'échelle obligatoire de `X_train` et de `X_test` pour assurer que le classificateur KNN fonctionne correctement

# Effectuée après la séparation des jeux d'entraînement et de test pour éviter que des informations concernant le second fuitent vers le premier

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 4. Rééquilibrage

In [ ]:
# Sous-échantillonnage avec `RandomUnderSampler`

from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state = 12)
X_train, y_train = rus.fit_resample(X_train, y_train)

In [ ]:
# Vérification de la longueur de `X_train`

len(X_train)

51712

## 5. Conversion

In [ ]:
# Création de la fonction artisanale `numpyfy`

def numpyfy(
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
    X_to_numpy: bool = True,
):
    """

    Cette fonction convertit en arrays les 4 jeux de données issus du
    découpage effectué par le splitter (train_test_split / TimeSeriesSplit)
    afin de les rendre compatibles avec `sktime`.

    Arguments :

        X_train (DataFrame) : données explicatives du jeu d'entraînement
        X_test (DataFrame) : données explicatives du jeu de test
        y_train (Series) : données cibles du jeu d'entraînement
        y_test (Series) : données cibles du jeu de test
        X_to_numpy (Boolean) :
            - précise si X_train et X_test doivent être convertis en arrays
            avant d'être remodelés
            - valeur par défaut : `True` (X_train et X_test sont des
            DataFrames)
            - si X_train et X_test sont déjà des arrays (notamment après
            transformation par un scaler, indiquer `False`)

    Retourne :

        X_train_np (array de 3 dimensions) : données explicatives du jeu
        d'entraînement, remodelées selon la structure suivante :

            - Dimension 1 (num_instances) : nombre d'INSTANCES de séries
            temporelles. Pour cette étude, dont l'unité temporelle est le
            jour, nous considérons 1 jour comme 1 instance.

            - Dimension 2 (num_variables) : nombre de VARIABLES EXPLICATIVES
            par instance de série temporelle.

            - Dimension 3 (length) : nombre de POINTS TEMPORELS observés par
            instance de série temporelle. Pour cette étude, comme l'unité
            temporelle est le jour, chaque instance de série temporelle
            correspond à 1 seul point temporel.

            (Référence :
            https://www.sktime.net/en/latest/examples/02_classification.html)

        X_test_np (array de 3 dimensions) : données explicatives du jeu de
        test, remodelées selon la structure ci-dessus.

        y_train_np (array de 1 dimension) : données cibles du jeu
        d'entraînement.

        y_test_np (array de 1 dimension) : données cibles du jeu de test.

    """

    ## Récupération des dimensions

    X_train_d1, X_train_d2 = X_train.shape
    X_test_d1, X_test_d2 = X_test.shape

    ## Conversion des DataFrames `X` en arrays de 3 dimensions

    if X_to_numpy is True:
        X_train_np = X_train.to_numpy().reshape(X_train_d1, X_train_d2, 1)
        X_test_np = X_test.to_numpy().reshape(X_test_d1, X_test_d2, 1)
    elif X_to_numpy is False:
        X_train_np = X_train.reshape(X_train_d1, X_train_d2, 1)
        X_test_np = X_test.reshape(X_test_d1, X_test_d2, 1)

    ## Conversion des Series `y` en arrays de 1 dimension

    y_train_np = y_train.to_numpy()
    y_test_np = y_test.to_numpy()

    ## Sortie des arrays convertis

    return X_train_np, X_test_np, y_train_np, y_test_np

In [ ]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test,
                                           X_to_numpy = False)

## 6. Modélisation

In [ ]:
# Importation de la classe `KNeighborsTimeSeriesClassifier`

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [ ]:
# Instanciation d'un modèle classificateur avec `n_neighbors = 1` et `distance = "dtw"`

clf_knn_ts = KNeighborsTimeSeriesClassifier(n_neighbors = 1, distance = "dtw")

In [ ]:
# Récupération des paramètres initiaux

clf_knn_ts.get_params()

{'algorithm': 'brute',
 'distance': 'dtw',
 'distance_mtype': None,
 'distance_params': None,
 'leaf_size': 30,
 'n_jobs': None,
 'n_neighbors': 1,
 'pass_train_distances': False,
 'weights': 'uniform'}

In [ ]:
# Entraînement du modèle

clf_knn_ts.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier()

In [ ]:
# Récupération des paramètres ajustés

clf_knn_ts.get_fitted_params()

{'classes': array([0, 1]),
 'fit_time': 6476,
 'knn_estimator': KNeighborsClassifier(algorithm='brute', metric='precomputed', n_neighbors=1),
 'n_classes': 2,
 'knn_estimator__classes': array([0, 1]),
 'knn_estimator__effective_metric': 'precomputed',
 'knn_estimator__effective_metric_params': {},
 'knn_estimator__n_features_in': 51712,
 'knn_estimator__n_samples_fit': 51712,
 'knn_estimator__outputs_2d': False}

In [ ]:
# Réalisation des prédictions

y_pred = clf_knn_ts.predict(X_test)

## 7. Évaluation

In [ ]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,12730,5389
1,1572,3773


In [ ]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.70      0.79     18119
           1       0.41      0.71      0.52      5345

    accuracy                           0.70     23464
   macro avg       0.65      0.70      0.65     23464
weighted avg       0.78      0.70      0.72     23464

